In [2]:
import pandas as pd 
df = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [4]:
df["Attrition"].value_counts()

Attrition
No     1233
Yes     237
Name: count, dtype: int64

In [5]:
# around 16% of employees left

In [6]:
# checking missing values for important columns
important_cols = [
"Attrition", "Age", "Gender", "MaritalStatus", "Department", "JobRole",
"JobLevel", "BusinessTravel", "DistanceFromHome", "OverTime",
"MonthlyIncome", "PercentSalaryHike", "StockOptionLevel",
"JobSatisfaction", "EnvironmentSatisfaction", "WorkLifeBalance",
"RelationshipSatisfaction", "TotalWorkingYears", "YearsAtCompany",
"YearsInCurrentRole", "YearsSinceLastPromotion", "YearsWithCurrManager"
]
df[important_cols].isnull().sum()


Attrition                   0
Age                         0
Gender                      0
MaritalStatus               0
Department                  0
JobRole                     0
JobLevel                    0
BusinessTravel              0
DistanceFromHome            0
OverTime                    0
MonthlyIncome               0
PercentSalaryHike           0
StockOptionLevel            0
JobSatisfaction             0
EnvironmentSatisfaction     0
WorkLifeBalance             0
RelationshipSatisfaction    0
TotalWorkingYears           0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int64

In [8]:
df.duplicated().sum() 
# checks duplicate values

np.int64(0)

In [10]:
# to check impossible values
df["Age"].min(),df["Age"].max()


(18, 60)

In [11]:
df ["MonthlyIncome"].min(), df["MonthlyIncome"].max()

(1009, 19999)

In [13]:
# removing of unnecessary columns

In [14]:
df_clean = df.drop(
    columns=["EmployeeNumber", "EmployeeCount", "Over18", "StandardHours"]
)


In [16]:
df_clean.shape # data size

(1470, 31)

In [28]:
import sqlite3 
conn = sqlite3.connect("hr_attrition.db")

In [29]:
df_clean.to_sql(
    "employees",
    conn,
    if_exists="replace",
    index=False
)

1470

In [27]:
import pandas as pd

pd.read_sql(
    "SELECT COUNT(*) AS total_employees FROM employees",
    conn
)
#verfying if table exists

,total_employees
0,1470


In [30]:
#analysis using sql
pd.read_sql(
    """
    SELECT Attrition, COUNT(*) AS count FROM employees GROUP BY Attrition """, conn)

,Attrition,count
0,No,1233
1,Yes,237


In [34]:
pd.read_sql(
    "PRAGMA table_info(employees);",conn)  #checks column included in sql

,cid,name,type,notnull,dflt_value,pk
0,0,Age,INTEGER,0,None,0
1,1,Attrition,TEXT,0,None,0
2,2,BusinessTravel,TEXT,0,None,0
3,3,DailyRate,INTEGER,0,None,0
4,4,Department,TEXT,0,None,0
5,5,DistanceFromHome,INTEGER,0,None,0
6,6,Education,INTEGER,0,None,0
7,7,EducationField,TEXT,0,None,0
8,8,EnvironmentSatisfaction,INTEGER,0,None,0
9,9,Gender,TEXT,0,None,0


In [35]:
pd.read_sql(
    """
    SELECT Department, Attrition, COUNT(*) AS count
    FROM employees
    GROUP BY Department, Attrition
    ORDER BY Department;
    """, conn )

,Department,Attrition,count
0,Human Resources,No,51
1,Human Resources,Yes,12
2,Research & Development,No,828
3,Research & Development,Yes,133
4,Sales,No,354
5,Sales,Yes,92


In [49]:
pd.read_sql(
    """
    SELECT 
        Department,
        COUNT(*) AS total_employees,
        SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS left_count,
        ROUND(
            100.0 * SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),
            2
        ) AS attrition_rate
    FROM employees
    GROUP BY Department;
    """,
    conn
)
# converted counts into rate

,Department,total_employees,left_count,attrition_rate
0,Human Resources,63,12,19.05
1,Research & Development,961,133,13.84
2,Sales,446,92,20.63


In [48]:
dept_attrition = pd.read_sql(
    """
    SELECT 
        Department,
        COUNT(*) AS total_employees,
        SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS left_count,
        ROUND(
            100.0 * SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),
            2
        ) AS attrition_rate
    FROM employees
    GROUP BY Department;
    """,
    conn
)
#used in power bi

Attrition rates vary significantly across departments. Certain departments show a higher percentage of employee exits, indicating department-specific workload, role pressure, or management challenges.

In [38]:
# checking if overtime effect
pd.read_sql(
    """
    SELECT OverTIme, Attrition, COUNT(*) AS count
    FROM employees GROUP BY OverTime, Attrition;""",conn)
    


,OverTime,Attrition,count
0,No,No,944
1,No,Yes,110
2,Yes,No,289
3,Yes,Yes,127


In [50]:
overtime_attrition = pd.read_sql(
    """
    SELECT 
        OverTime,
        Attrition,
        COUNT(*) AS employee_count
    FROM employees
    GROUP BY OverTime, Attrition;
    """,
    conn
)
#used for Power BI

Employees who work overtime show significantly higher attrition compared to those who do not. This suggests that excessive workload may be a key factor driving employees to leave the company

In [40]:
# effects by job role
pd.read_sql(
    """
    SELECT JobRole, Attrition, COUNT(*) AS count FROM employees
    GROUP BY JobRole, Attrition
    ORDER BY count DESC; """, conn)

,JobRole,Attrition,count
0,Sales Executive,No,269
1,Research Scientist,No,245
2,Laboratory Technician,No,197
3,Manufacturing Director,No,135
4,Healthcare Representative,No,122
5,Manager,No,97
6,Research Director,No,78
7,Laboratory Technician,Yes,62
8,Sales Executive,Yes,57
9,Sales Representative,No,50


In [52]:
pd.read_sql(
    """
    SELECT 
        JobRole,
        COUNT(*) AS total_employees,
        SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS left_count,
        ROUND(
            100.0 * SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),
            2
        ) AS attrition_rate
    FROM employees
    GROUP BY JobRole
    ORDER BY attrition_rate DESC;
    """,
    conn
)
# converted into rate

,JobRole,total_employees,left_count,attrition_rate
0,Sales Representative,83,33,39.76
1,Laboratory Technician,259,62,23.94
2,Human Resources,52,12,23.08
3,Sales Executive,326,57,17.48
4,Research Scientist,292,47,16.10
5,Manufacturing Director,145,10,6.90
6,Healthcare Representative,131,9,6.87
7,Manager,102,5,4.90
8,Research Director,80,2,2.50


In [ ]:
jobrole_attrition = pd.read_sql(
    """
    SELECT 
        JobRole,
        COUNT(*) AS total_employees,
        SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS left_count,
        ROUND(
            100.0 * SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),
            2
        ) AS attrition_rate
    FROM employees
    GROUP BY JobRole
    ORDER BY attrition_rate DESC;
    """,
    conn
)
#used for Power BI

Attrition is not evenly distributed across job roles. Certain roles exhibit significantly higher attrition rates, indicating role-specific stress, workload imbalance, or limited career growth opportunities

In [42]:
# effecting by average montly income
pd.read_sql(
    """
    SELECT Attrition, AVG(MonthlyIncome) AS avg_income
    FROM employees GROUP BY Attrition; """, conn )

,Attrition,avg_income
0,No,6832.739659
1,Yes,4787.092827


In [53]:
pd.read_sql(
    """
    SELECT
        CASE
            WHEN MonthlyIncome < 5000 THEN 'Low Income'
            WHEN MonthlyIncome BETWEEN 5000 AND 10000 THEN 'Medium Income'
            ELSE 'High Income'
        END AS income_group,
        Attrition,
        COUNT(*) AS employee_count
    FROM employees
    GROUP BY income_group, Attrition
    ORDER BY income_group;
    """,
    conn
)
#income distribution

,income_group,Attrition,employee_count
0,High Income,No,256
1,High Income,Yes,25
2,Low Income,No,586
3,Low Income,Yes,163
4,Medium Income,No,391
5,Medium Income,Yes,49


In [54]:
income_attrition = pd.read_sql(
    """
    SELECT
        CASE
            WHEN MonthlyIncome < 5000 THEN 'Low Income'
            WHEN MonthlyIncome BETWEEN 5000 AND 10000 THEN 'Medium Income'
            ELSE 'High Income'
        END AS income_group,
        Attrition,
        COUNT(*) AS employee_count
    FROM employees
    GROUP BY income_group, Attrition
    ORDER BY income_group;
    """,
    conn
)
# used in Power BI

Employees in lower income groups show noticeably higher attrition compared to medium and high income groups. This indicates that compensation plays a significant role in employee retention.

In [55]:
# average years of experince at company
pd.read_sql(
    """
    SELECT 
        Attrition,
        ROUND(AVG(YearsAtCompany), 2) AS avg_years_at_company,
        ROUND(AVG(TotalWorkingYears), 2) AS avg_total_working_years
    FROM employees
    GROUP BY Attrition;
    """,
    conn
)


,Attrition,avg_years_at_company,avg_total_working_years
0,No,7.37,11.86
1,Yes,5.13,8.24


In [57]:
# groups of different years

pd.read_sql(
    """
    SELECT
        CASE
            WHEN YearsAtCompany < 3 THEN '0–2 years'
            WHEN YearsAtCompany BETWEEN 3 AND 7 THEN '3–7 years'
            ELSE '8+ years'
        END AS experience_group,
        Attrition,
        COUNT(*) AS employee_count
    FROM employees
    GROUP BY experience_group, Attrition
    ORDER BY experience_group;
    """,
    conn
)


,experience_group,Attrition,employee_count
0,0–2 years,No,240
1,0–2 years,Yes,102
2,3–7 years,No,520
3,3–7 years,Yes,80
4,8+ years,No,473
5,8+ years,Yes,55


In [58]:
experience_attrition = pd.read_sql(
    """
    SELECT
        CASE
            WHEN YearsAtCompany < 3 THEN '0–2 years'
            WHEN YearsAtCompany BETWEEN 3 AND 7 THEN '3–7 years'
            ELSE '8+ years'
        END AS experience_group,
        Attrition,
        COUNT(*) AS employee_count
    FROM employees
    GROUP BY experience_group, Attrition
    ORDER BY experience_group;
    """,
    conn
)
# used for Power BI

Employees with fewer years at the company show significantly higher attrition. Attrition decreases as employee tenure increases, highlighting the importance of onboarding, early engagement, and career development for new hires.

In [59]:
overtime_attrition.to_csv("overtime_attrition.csv", index=False)

In [60]:
dept_attrition.to_csv("department_attrition.csv", index=False)

In [62]:
jobrole_attrition = pd.read_sql(
    """
    SELECT 
        JobRole,
        COUNT(*) AS total_employees,
        SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS left_count,
        ROUND(
            100.0 * SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),
            2
        ) AS attrition_rate
    FROM employees
    GROUP BY JobRole
    ORDER BY attrition_rate DESC;
    """,
    conn
)


In [63]:
jobrole_attrition.head()


,JobRole,total_employees,left_count,attrition_rate
0,Sales Representative,83,33,39.76
1,Laboratory Technician,259,62,23.94
2,Human Resources,52,12,23.08
3,Sales Executive,326,57,17.48
4,Research Scientist,292,47,16.10


In [64]:
jobrole_attrition.to_csv("jobrole_attrition.csv", index=False)


In [65]:
income_attrition.to_csv("income_attrition.csv", index=False)

In [66]:
experience_attrition.to_csv("experience_attrition.csv", index=False)

In [67]:
df_clean

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,2,Female,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,3,Male,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,4,Male,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,4,Female,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,Male,...,3,4,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,3,Male,...,3,3,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,4,Male,...,3,1,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,2,Male,...,4,2,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,4,Male,...,3,4,0,17,3,2,9,6,0,8


In [69]:
df_clean.to_csv("clean_hr_attrition.csv",index=False)